# Vector Search
## Using langchain for utilizing OpenAI embedding model

In [19]:
! pip install langchain
! pip install tiktoken

# saving dataframe as .xlsx format
! pip install openpyxl

In [20]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
# from openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from langchain.vectorstores import FAISS

import ast


### import dataset

In [21]:
spider_df = pd.read_csv('./git_pull/GS_BKMS2/NL2SQL/rawdata/SPIDER_SELECTED.csv')
ddl_df = pd.read_csv('./git_pull/GS_BKMS2/NL2SQL/rawdata/DDL_SELECTED.csv')

In [22]:
spider_df = spider_df.loc[:,['original_idx', 'db_id', 'question', 'query']]

In [23]:
spider_df.head(3)

,original_idx,db_id,question,query
0,11,department_management,How many departments are led by heads who are ...,SELECT count(*) FROM department WHERE departme...
1,13,department_management,List the states where both the secretary of 'T...,SELECT T3.born_state FROM department AS T1 JOI...
2,14,department_management,Which department has more than 1 head at a tim...,"SELECT T1.department_id , T1.name , count(*)..."


### embedding

In [24]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# text-embedding-ada-002 embedding model from OpenAI -> results in 1,536 dimensional vector
embeddings = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    openai_api_key = OPENAI_API_KEY 
)

In [25]:
ddl_df['CREATE_cleaned'] = ddl_df['CREATE'].apply(lambda x: x.replace('\n', ' ').replace('\t', ' ')
                                                            .replace('CREATE TABLE','').replace('IF NOT EXISTS','')
                                                            .replace('create table','').replace('if not exists',''))

# embedding CREATE TABLE queries
ddl_df['ddl_vec'] = embeddings.embed_documents(texts = ddl_df['CREATE_cleaned'])
# embedding qNL (natural language questions)
spider_df['qnl_vec'] = embeddings.embed_documents(texts = spider_df['question'])

# save the resulted embedding vectors to save API cost
ddl_df.to_csv('DDL_embed.csv', index = False)
spider_df.to_csv('SPIDER_embed.csv', index = False)

In [26]:
#### import embedding vectors
ddl_emb = pd.read_csv('DDL_embed.csv')
spider_emb = pd.read_csv('SPIDER_embed.csv')

# convert csv's string representations of embedding vectors into original type after reading the csv file (plain text)
ddl_emb['ddl_vec'] = ddl_emb['ddl_vec'].apply(ast.literal_eval)
spider_emb['qnl_vec'] = spider_emb['qnl_vec'].apply(ast.literal_eval)

In [27]:
# check whether the vectors are in expected dimensions
print(len(spider_df.loc[0, 'qnl_vec']), len(ddl_df.loc[0, 'ddl_vec']))

1536 1536


### compute similarity matrix

#### qNL ↔︎ DDL

In [28]:
qnl_vec = np.array(spider_emb['qnl_vec'].tolist())
ddl_vec = np.array(ddl_emb['ddl_vec'].tolist())

similarity_matrix = cosine_similarity(qnl_vec, ddl_vec)
# similarity_matrix = 1 - euclidean_distances(spider_vec, ddl_vec)

# (224: # of natural language questions, 434: # of ddl)
print(similarity_matrix.shape)

(224, 434)


In [29]:
result = []

# iterate through each query to find the top 5 similar CREATE TABLE queries
# e.g., 0, 'How many departments are led by heads who are not mentioned?'
for i, qnl in enumerate(spider_emb['question']):
    original_idx, db_id_query = spider_emb.loc[i, ['original_idx', 'db_id']]
    
    similarities = similarity_matrix[i] # 434 CREATE queries

    ddl_similarities = [(ddl_emb.loc[j, 'db_id'], ddl_emb.loc[j, 'table_name'], ddl_emb.loc[j, 'CREATE'], sim) for j, sim in enumerate(similarities)]

    # Sorting based on similarity score and selecting the top 5
    relevent_ddl = sorted(ddl_similarities, key=lambda x: x[3], reverse=True)[:5]

    # Adding the results to the result list
    for db_id_ddl, table_name, create, sim in relevent_ddl:
        result.append([original_idx, qnl, db_id_query, db_id_ddl, table_name, create, sim])

# Creating the result DataFrame
result_df = pd.DataFrame(result, columns=['original_idx', 'qNL', 'db_id_query', 'db_id_ddl', 'table_name', 'CREATE', 'similarities'])
result_df

,original_idx,qNL,db_id_query,db_id_ddl,table_name,CREATE,similarities
0,11,How many departments are led by heads who are ...,department_management,department_management,department,CREATE TABLE IF NOT EXISTS department (\n\tDep...,0.753460
1,11,How many departments are led by heads who are ...,department_management,department_management,management,CREATE TABLE IF NOT EXISTS management (\n\tDep...,0.751721
2,11,How many departments are led by heads who are ...,department_management,hr_1,departments,CREATE TABLE IF NOT EXISTS departments (\n DE...,0.751448
3,11,How many departments are led by heads who are ...,department_management,department_store,Departments,CREATE TABLE Departments (\ndepartment_id INTE...,0.742923
4,11,How many departments are led by heads who are ...,department_management,hr_1,employees,CREATE TABLE IF NOT EXISTS employees (\n EMPL...,0.729376
...,...,...,...,...,...,...,...
1115,7955,Which dogs are of the rarest breed? Show their...,dog_kennels,dog_kennels,Breeds,CREATE TABLE Breeds ( breed_code VARCHAR(20) P...,0.744857
1116,7955,Which dogs are of the rarest breed? Show their...,dog_kennels,dog_kennels,Treatments,CREATE TABLE Treatments ( treatment_id INTEGER...,0.741661
1117,7955,Which dogs are of the rarest breed? Show their...,dog_kennels,dog_kennels,Dogs,CREATE TABLE Dogs ( dog_id INTEGER PRIMARY KEY...,0.727230
1118,7955,Which dogs are of the rarest breed? Show their...,dog_kennels,pets_1,Pets,"CREATE TABLE Pets ( PetID INTEGER PRIMARY KEY,...",0.705754


In [30]:
# out of the top 5 similar CREATE TABLE queries, 2.7 db_ids on average do not match the true db_id
result_df['mismatch'] = (result_df['db_id_query'] != result_df['db_id_ddl']).astype(int)
mismatch_count = result_df.groupby('original_idx')['mismatch'].sum()

print(np.average(mismatch_count))

2.700892857142857


#### qNL ↔︎ qNL

In [31]:
q_similarity_matrix = cosine_similarity(qnl_vec, qnl_vec)
# q_similarity_matrix = 1 - euclidean_distances(qnl_vec, qnl_vec)
print(q_similarity_matrix.shape)

(224, 224)


In [32]:
result_q = []

for i, qnl in enumerate(spider_emb['question']):
    original_idx, db_id_query = spider_emb.loc[i, ['original_idx', 'db_id']]
    
    similarities = q_similarity_matrix[i]

    qnl_similarities = [(spider_emb.loc[j, 'db_id'], spider_emb.loc[j, 'question'], sim) for j, sim in enumerate(similarities)]

    similar_qnl = sorted(qnl_similarities, key=lambda x: x[2], reverse=True)[:5]

    for db_id_similar, qnl_similar, sim in similar_qnl:
        result_q.append([original_idx, db_id_query, qnl, qnl_similar, db_id_similar, sim])

result_df_q = pd.DataFrame(result_q, columns=['original_idx', 'db_id_query', 'qNL', 'qNL_similar', 'db_id_similar', 'similarities'])
result_df_q

,original_idx,db_id_query,qNL,qNL_similar,db_id_similar,similarities
0,11,department_management,How many departments are led by heads who are ...,How many departments are led by heads who are ...,department_management,1.000000
1,11,department_management,How many departments are led by heads who are ...,Which department has more than 1 head at a tim...,department_management,0.878852
2,11,department_management,How many departments are led by heads who are ...,How many students and instructors are in each ...,college_2,0.828406
3,11,department_management,How many departments are led by heads who are ...,How many different professors are there for th...,college_1,0.823827
4,11,department_management,How many departments are led by heads who are ...,What is the name of the department that has th...,college_1,0.820650
...,...,...,...,...,...,...
1115,7955,dog_kennels,Which dogs are of the rarest breed? Show their...,Which dogs are of the rarest breed? Show their...,dog_kennels,1.000000
1116,7955,dog_kennels,Which dogs are of the rarest breed? Show their...,List the names of the dogs of the rarest breed...,dog_kennels,0.968927
1117,7955,dog_kennels,Which dogs are of the rarest breed? Show their...,Which owner has paid for the most treatments o...,dog_kennels,0.859423
1118,7955,dog_kennels,Which dogs are of the rarest breed? Show their...,Tell me the owner id and last name of the owne...,dog_kennels,0.856344


In [33]:
# out of the top 5 similar CREATE TABLE queries, 2.1 db_ids on average do not match the true db_id
result_df_q['mismatch'] = (result_df_q['db_id_query'] != result_df_q['db_id_similar']).astype(int)
mismatch_count = result_df_q.groupby('original_idx')['mismatch'].sum()

print(np.average(mismatch_count))

2.1026785714285716


In [34]:
result_df_q.to_excel('similar_questions.xlsx')

## Using FAISS (Facebook AI Similarity Search)

https://jfan001.medium.com/improving-vector-search-by-converting-documents-to-question-answer-pairs-4ded4769326a

In [35]:
qs = []
for q in spider_df['question']:
    qs.append(Document(page_content=q))

In [36]:
embeddings = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    openai_api_key = OPENAI_API_KEY 
)

qa_db = FAISS.from_documents(qs, embeddings)

In [37]:
# returned distance score is L2 distance
# https://python.langchain.com/docs/integrations/vectorstores/faiss
results = qa_db.similarity_search_with_score(spider_df['question'][0], k=5)

In [38]:

df = pd.DataFrame(results, columns=["question", "distance"])
df

,question,distance
0,page_content='How many departments are led by ...,0.000000
1,page_content='Which department has more than 1...,0.242296
2,page_content='How many students and instructor...,0.343188
3,page_content='How many different professors ar...,0.352416
4,page_content='What is the name of the departme...,0.358640
